In [6]:
# 라이브러리
import requests, xmltodict, json
import pandas as pd

### 1) 서울특별시_정류소정보조회 서비스  
* 7_getStaionsByPosList 활용
* **input: tmX, tmY, radius => output: stId, stationName, arsId**
* https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15000303

In [7]:
# 사용자의 위치를 받아서 tmX, tmY 변수에 지정.
tmX = 126.948105
tmY = 37.540897
radius = 200 # 범위 (넓히면 여러 정류장 인식 됨.)

key = "AT98N5LWRAir0I67tVgrf6Vfnio9LCMcwusSbOjmdkEpSOGyobdyAq9cb41G6O4pgTp6Jcmpv8e87bplMNY7tQ%3D%3D"
url = f'http://ws.bus.go.kr/api/rest/stationinfo/getStationByPos?ServiceKey={key}&tmX={tmX}&tmY={tmY}&radius={radius}'
 
content = requests.get(url).content
dict = xmltodict.parse(content)

# 첫번째 정류장이라 설정 (음성으로 "현재 인식된 정류장은 A 정류장 입니다." 라고 전해주기)
jsonString = json.dumps(dict['ServiceResult']['msgBody']['itemList'][0]['stationNm'], ensure_ascii=False)
jsonObj = json.loads(jsonString)
# print(jsonObj)

# 향후 사용: 정류소 고유 번호, 정류소명 추출
target_stId = int(dict['ServiceResult']['msgBody']['itemList'][0]['stationId'])
target_stationName = str(dict['ServiceResult']['msgBody']['itemList'][0]['stationNm'])
target_arsId = str(dict['ServiceResult']['msgBody']['itemList'][0]['arsId'])
print(target_stId) # int
print(target_stationName)
print(target_arsId) # str

113000001
마포역
14001


### 2) Excel Data를 통해 원하는 데이터 추출
* Excel Data: 1.busnumber_to_busRouteid , 2.stationName_to_ord 
* 다음 API 이용하는데 필요한 busRouteId, ord 추출하기 위함.
* **문제: 정류소 ord가 2020.9.2 기준이므로 이후 변경된 노선은 결과값 출력이 안됨**

In [8]:
data1 = pd.read_csv('data/busnumber_to_busRouteid.csv', encoding='utf-8')
data2 = pd.read_csv('data/stationName_to_ord.csv', encoding='utf-8')
display(data1.head())
display(data2.head())

# 음성인식을 통해 사용자가 160번 탄다고 가정했음.
target_bus = '160' # str 형태로 해야됨. -> 엑셀이 str 형태

# 조건에 맞는 버스 아이디 추출
target_busRouteId = data1[data1['busNumber'] == target_bus].iloc[0]['busRouteId']
print(target_busRouteId) # int

# 조건에 맞는 ord 추출
target_ord = data2[(data2['stationName'] == target_stationName) & (data2['busNumber'] == target_bus) & (data2['stationNumber'] == int(target_arsId))].iloc[0]['ord']
print(target_ord) # int

,busNumber,busRouteId
0,0017,100100124
1,02,100100001
2,03,100100002
3,05,100100003
4,100,100100549


,busNumber,type,ord,stationName,stationNumber
0,0017,지선,1,청암자이아파트,3689
1,0017,지선,2,청암동강변삼성아파트,3298
2,0017,지선,3,청심경로당,3321
3,0017,지선,4,원효2동주민센터,3304
4,0017,지선,5,산천동,3306


100100033
78


### 3) 서울특별시_버스도착정보조회 서비스
* 2_getArrInfoByRouteList
* **input: stId, busRouteId, ord => output: arrmsg1, arrmsg2** + 이 외 필요한 것들 추가
* https://www.data.go.kr/iim/api/selectAPIAcountView.do

In [9]:
#서울특별시_버스도착정보조회 서비스 中 2_getArrInfoByRouteList
key = 'kNSQvU5WeosgTXwCx1mTthdz93%2BlLXHKA7ZtzbuNArBuUVVP4akW5xsfp6R5JYuMH106DwcuJRTqXJHI4q%2BNjA%3D%3D'
url = 'http://ws.bus.go.kr/api/rest/arrive/getArrInfoByRoute?ServiceKey={}&stId={}&busRouteId={}&ord={}'.format(key, target_stId, target_busRouteId, target_ord)

content = requests.get(url).content
dict = xmltodict.parse(content)

busstation = json.dumps(dict['ServiceResult']['msgBody']['itemList']['stNm'], ensure_ascii = False)
arrival = json.dumps(dict['ServiceResult']['msgBody']['itemList']['arrmsg1'], ensure_ascii = False)
arrival2 = json.dumps(dict['ServiceResult']['msgBody']['itemList']['arrmsg2'], ensure_ascii = False)
busplainnum = json.dumps(dict['ServiceResult']['msgBody']['itemList']['plainNo1'], ensure_ascii = False)

jsonbusstation = json.loads(busstation)
jsonarrival = json.loads(arrival)
jsonsarrival2 = json.loads(arrival2)
jsonsbusplainnum = json.loads(busplainnum)

print('정류소명: ' + jsonbusstation)
print('첫 번째 버스: ' + jsonarrival)
print('두 번째 버스: ' + jsonsarrival2)
print('도착 예정 버스 차량 번호: ' + jsonsbusplainnum)

정류소명: 마포역
첫 번째 버스: 곧 도착
두 번째 버스: 6분47초후[1번째 전]
도착 예정 버스 차량 번호: 서울70사6732


#### **필요하다면 다음 정류장 이름도 제시해도됨 -> 방향이 맞는지 다시 한번 확인**
-----------------

### <문제 해결 방안>
#### 1) 서울특별시_노선정보조회 서비스 1_getStaionsByRouteList 이용하여 seq를 ord로 저장하여 직접 DB 구축
#### 2) 서울특별시_노선정보조회 서비스 1_getStaionsByRouteList 이용하여 실시간 API 확인하여 seq 추출 -> 계산량 많을듯

* 현재로써는 데이터 수집과정은 번거로울 수 있으나 1안 이용해서 직접 DB 구축하는게 좋아보임.  